In [18]:
# This notebook will allow you to enter a city of your choice from anywhere in the world and receive the current weather details for that location.

In [19]:
!pip install --user requests


In [20]:
# -*- coding: utf-8 -*-

import requests
import json
#exclude_list = ["icon", "id"]  

api_key = ""
with open('.security.json') as f:
    config_seqdata = json.load(f)
    api_key = config_seqdata['api_key']

In [21]:
location_settings = { 
    "coord":["lon","lat"],    
    "sys": ["type", "id", "country", "sunrise", "sunset"],
    "timezone": "timezone",
    "units": "metric"
}

# TBD: display icon from url : https://openweathermap.org/img/wn/{icon}}@2x.png
# TBD : display timestamp , timezone , sunrise and sunset in human readable format
# TBD: weather_api  should be used to filter printed details on weather
weather_api = {        
        "weather":["main", "description", "icon"],
        "main":["temp", "feels_like", "temp_min", "temp_max", "pressure", "humidity"],
        "visibility": ["visibility"],
        "wind": ["speed", "deg"],
        "clouds": ["all"],
        "rain": ["1h"],
        "snow": ["1h"],
        "dt": ["dt"],   
}

weather_api_units = { "main": 
                        {
                            "temp": "°C", 
                            "feels_like": "°C", 
                            "temp_min": "°C", 
                            "temp_max": "°C", 
                            "pressure": "hPa", 
                            "humidity": "%"
                        },                      
                     "visibility": "meters", 
                     "wind": {"speed": "m/s" , "deg": "°" , "gust":"m/s" }, 
                     "rain": {"1h": "mm"}, 
                     "snow": {"1h": "mm"}}
                     
# f.e. location settings : { "London" : { "lat": 51.5074, "lon": 0.1278, "timezone": "Europe/London" } }
locations = {}
weather = {}

def print_weather_with_units(weather_dict, units_dict):
    for key, value in weather_dict.items():
        if isinstance(value, dict):
            print(f"{key}:")
            print_weather_with_units(value, units_dict.get(key, {}))
        else:
            unit = units_dict.get(key, "")
            print(f"{key}: {value} {unit}")
    print()

def parse_openweather_response(json_str):
    
    '''
        This function parses weather information from the JSON response.
        The parsed weather information is stored in the `weather` dictionary, which contains various weather components such as 
        temperature, humidity, wind speed, etc. 
        The global `locations` dictionary is also updated with the city's information, including its coordinates, timezone, and units.
        Finally, the function calls the `print_weather_with_units()` function to print the weather information with appropriate units ,
        as configured. 
    '''
    data = json.loads(json_str)
    city = data["name"]
    locations[city] = {}
    locations[city]['units'] = "metric" # later take from the settings !!!!
        
    for key, value in data.items():        
        for weather_key , weather_value in weather_api.items():
            if weather_key  == key:
                if isinstance(value, list):                                                                    
                    weather[weather_key] = value[0] # only first weather item
                else:
                    weather[weather_key] = value            

        for location_key , location_value in location_settings.items()  :
            if location_key == key:                      
                for item in  location_settings[location_key]:                                        
                    if isinstance(value, str) or isinstance(value, int):
                        locations[city][str(key)] = value
                    else:                        
                        locations[city][item] = value[item]
                                    
    print_weather_with_units(weather, weather_api_units)    
    print()

def weather_checker(city_name):        

    '''
        The function `weather_checker(city_name)` is responsible for checking the weather information for a given city. 
        It takes a `city_name` parameter as input and uses the OpenWeatherMap API to retrieve the weather data for that city.
        The function first constructs the API URL using the provided `city_name` and the API key. It then sends a GET request
        to the API and receives the response. The response is checked for any errors, and if there are no errors, 
        
        The function returns `True` if the weather information was successfully retrieved, and `False` otherwise.
        Overall, the `weather_checker(city_name)` function provides a convenient way to check the weather for a specific city 
        using the OpenWeatherMap API.
    '''    
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name.lower()}&appid={api_key}&units=metric"
    response = requests.get(url)    
    data = response.json()    
    if data["cod"] != 200:
        print("Error: ", data["message"])
        return False 
    
    print(f"=========== Weather in {city_name} ================")
    parse_openweather_response(response.text)                  
    return data["cod"] == 200


The function `weather_checker(city_name)` is responsible for checking the weather information for a given city. It takes a `city_name` parameter as input and uses the OpenWeatherMap API to retrieve the weather data for that city.

The function first constructs the API URL using the provided `city_name` and the API key. It then sends a GET request to the API and receives the response. The response is checked for any errors, and if there are no errors, the weather information is parsed from the JSON response.

The parsed weather information is stored in the `weather` dictionary, which contains various weather components such as temperature, humidity, wind speed, etc. The `locations` dictionary is also updated with the city's information, including its coordinates, timezone, and units.

Finally, `print_weather_with_units()` function is called to print the weather information contained in the above `weather` dictionary , with all appropriate units. The function returns `True` if the weather information was successfully retrieved, and `False` otherwise.

Overall, the `weather_checker(city_name)` function provides a convenient way to check the weather for a specific city using the OpenWeatherMap API.

In [22]:
weather_checker("Jerusalem")
weather_checker("Tokio")
weather_checker("New York")   
weather_checker("Detroit")


if locations.__sizeof__() > 0:  
    print(f"=========== Locations settings \n {json.dumps(locations, indent=4)} ")
else:
    # Main loop to get city names from user input
    while True:
        city = input("Enter a city name (or 'exit' to quit): ")
        if city.lower() == "exit" or city.lower() == "quit":
            break
        else:        
            if len(city) == 0:            
                print("You must enter a city name.")
                continue
            elif not weather_checker(city):
                print("Information about the city was not found. Please try again.")
                continue    
        

=========== Weather in Jerusalem ================
weather:
id: 800 
main: Clear 
description: clear sky 
icon: 01n 

main:
temp: 19.71 °C
feels_like: 19.49 °C
temp_min: 17.85 °C
temp_max: 20.37 °C
pressure: 1010 hPa
humidity: 67 %

visibility: 10000 meters
wind:
speed: 5.36 m/s
deg: 319 °
gust: 8.94 m/s

clouds:
all: 0 

dt: 1716655120 


=========== Weather in Tokio ================
weather:
id: 803 
main: Clouds 
description: broken clouds 
icon: 04n 

main:
temp: 18.98 °C
feels_like: 18.5 °C
temp_min: 18.12 °C
temp_max: 19.33 °C
pressure: 1016 hPa
humidity: 60 %

visibility: 10000 meters
wind:
speed: 2.06 m/s
deg: 90 °

clouds:
all: 75 

dt: 1716655628 


=========== Weather in New York ================
weather:
id: 800 
main: Clear 
description: clear sky 
icon: 01d 

main:
temp: 26.01 °C
feels_like: 26.01 °C
temp_min: 21.97 °C
temp_max: 29.73 °C
pressure: 1013 hPa
humidity: 37 %

visibility: 10000 meters
wind:
speed: 5.14 m/s
deg: 50 °

clouds:
all: 0 

dt: 1716655505 


=========